In [ ]:
# Start writing code here...
import numpy as np 
import pandas as pd
import collections
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import tqdm

## Dataset

In [ ]:
class YelpPolarity(Dataset):
    def __init__(self, csv_path):
        self.data_frame = pd.read_csv(csv_path)
    
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        data_instance = self.data_frame.iloc[idx]
        inp_string = data_instance[1]
        inp_string = inp_string[:min(char_len, len(inp_string))]
        if "A" not in alphabet: inp_string = inp_string.lower()
            
        X_lis = [len(alphabet)] * char_len
        X_lis[:len(inp_string)] = [char_map.get(char, len(alphabet)) for char in inp_string]
        
        X = torch.LongTensor(X_lis)
        y = torch.FloatTensor([data_instance[0]])
        return X, y

In [ ]:
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’’’/\|_@#$%ˆ&*˜‘+-=<>()[]{}"
char_map = {char:idx for idx, char in enumerate(alphabet)}
def generate_embedding_matrix(size=len(alphabet)):
    return torch.vstack([torch.eye(size), torch.zeros(size)])
batch_size = 2048
char_len = 1014
dset = YelpPolarity('train.csv')
train_loader = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, num_workers=2)

loader : data loaded training samples \
in_feats: input features \
val_data_loader: validation data loader



LSTM

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, ntoken, ninp, dim_hid, nlayers, nout, dropout=0.3):
        super(LSTMClassifier, self).__init__()
        self.nout = nout
        self.nlayers = nlayers
        self.dim_hid = dim_hid

        self.drop = nn.Dropout(dropout)

        self.embedding = nn.Embedding(ntoken, ninp)
        self.lstm = nn.LSTM(ninp, dim_hid, nlayers, 
                            dropout=dropout, 
                            batch_first=True)

        self.fc = nn.Linear(dim_hid, nout)
        self.sig = nn.Sigmoid()
    
    def forward(self, inp, hidden):
        bsize = inp.size[0]
        emb_out = self.embedding(inp)
        lstm_out, hidden = self.lstm(emb_out, hidden)

        lstm_out = lstm_out.contiguous().view(-1, self.dim_hid)

        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sig(out)

        out = out.view(bsize, -1)
        out = out[:,-1]

        return out, hidden
    
    def init_hidden(self, bsize):
        w = next(self.parameters()).data
        return (weight.new_zeros(self.nlayers, bsize, self.dim_hid),
                weight.new_zeros(self.nlayers, bsize, self.dim_hid))

In [ ]:
net = LSTMClassifier(char_len, ninp=char_len,
                     dim_hid=256, nlayers=2, nout=2)

In [ ]:
corpus = ['Hey how are you', 'Watcha doing', 'Sup']
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus)

## Bag of words

In [ ]:
class Model(nn.Module):
  def __init__(self, in_feats):
    super(Model, self).__init__()
    self.dropout = nn.Dropout(0.1)
    self.linear1 = nn.Linear(in_feats, 32)
    self.linear2 = nn.Linear(32, 1)
  def forward(self, X):
    ans = self.dropout(self.linear1(X))
    ans = self.linear2(ans)
    return ans

In [ ]:
def run(model, loader, optimizer, criterion, run_type="train"):
    if run_type == "train":
        model.train()
    else:
        model.eval()
    model.train()
    running_loss = 0
    total_iters = len(loader)
    for (corpus, labels) in loader:
        corpus = corpus.type(torch.float)
        labels = labels.type(torch.float)
        outputs = model(corpus)
        # outputs = outputs.squeeze()
        loss = criterion(outputs.float(), labels.float())
        running_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mean_loss = running_loss/total_iters
    return mean_loss

In [ ]:
def output_metrics(data_loader):
    optimizer = optimizer = optim.Adam(model_without_dropout.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()
    outs,lbl = eval(model, data_loader, optimizer, criterion)
    print("MSE: ", mean_squared_error(outs, lbl))
    return

In [ ]:
model = Model(1014)
optimizer = optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
for i in range(0, 200):
    print("Iter:", i)
    run(model, train_loader, optimizer, criterion)

Iter: 0
Exception ignored in: <function tqdm.__del__ at 0x7f3ca9f135f0>
Traceback (most recent call last):
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/tqdm/std.py", line 1147, in __del__
    self.close()Exception ignored in: 
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/tqdm/notebook.py", line 286, in close
<function tqdm.__del__ at 0x7f3ca9f135f0>    self.disp(bar_style='danger', check_delay=False)

AttributeErrorTraceback (most recent call last):
:   File "/shared-libs/python3.7/py/lib/python3.7/site-packages/tqdm/std.py", line 1147, in __del__
'tqdm_notebook' object has no attribute 'disp'    self.close()

  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/tqdm/notebook.py", line 286, in close
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3ca9d50830>    self.disp(bar_style='danger', check_delay=False)
Traceback (most recent call last):

  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/torch/ut

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f80c269c-dad1-4c63-bf97-6d9868e127ca' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>